In [1]:
from finbert.finbert import predict

In [2]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/erikivanov/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
bertmodel = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert",cache_dir=None, num_labels=3)

Downloading: 100%|██████████| 758/758 [00:00<00:00, 565kB/s]
Downloading: 100%|██████████| 418M/418M [01:29<00:00, 4.89MB/s] 


In [5]:
p = predict('Stocks rallied and the British pound gained.', bertmodel)
p

Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 31.2kB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 362kB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 832kB/s] 
Downloading: 100%|██████████| 455k/455k [00:00<00:00, 1.32MB/s]
01/18/2022 05:02:56 - INFO - finbert.utils -   *** Example ***
01/18/2022 05:02:56 - INFO - finbert.utils -   guid: 0
01/18/2022 05:02:56 - INFO - finbert.utils -   tokens: [CLS] stocks rallied and the british pound gained . [SEP]
01/18/2022 05:02:56 - INFO - finbert.utils -   input_ids: 101 15768 24356 1998 1996 2329 9044 4227 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
01/18/2022 05:02:56 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
01/18/2022 05:02:56 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

,sentence,logit,prediction,sentiment_score
0,Stocks rallied and the British pound gained.,"[0.8983616, 0.03447347, 0.06716488]",positive,0.863888


In [6]:
import pandas as pd

def predict_finbert(headline: str) -> pd.core.frame.DataFrame:
  return predict(headline, bertmodel)

In [7]:
headlines = pd.read_csv('headlines_sp500f.csv')
headlines.head()

,title,Date
0,Citi's top stocks for big dividend growth in 2...,2021-12-18
1,"Nasdaq, S&P 500 and Dow Jones all fall as rate...",2021-12-17
2,SocGen's Edwards says fear the FAANG falter in...,2021-12-17
3,"Nasdaq, S&P 500 drop as tech weakens, Dow Jone...",2021-12-16
4,"Dow Jones, S&P 500, Nasdaq rally, even as Powe...",2021-12-15


In [8]:
headlines_sentiment_analysis_result = {
    'date': [],
    'score': []
}

In [10]:
for i in headlines.index:
  score = predict_finbert(headlines['title'][i])['sentiment_score']

  headlines_sentiment_analysis_result['date'].append(headlines['Date'][i])
  headlines_sentiment_analysis_result['score'].append(score)

01/18/2022 05:04:33 - INFO - finbert.utils -   *** Example ***
01/18/2022 05:04:33 - INFO - finbert.utils -   guid: 0
01/18/2022 05:04:33 - INFO - finbert.utils -   tokens: [CLS] ci ##ti ' s top stocks for big divide ##nd growth in 202 ##2 : alpha tactics [SEP]
01/18/2022 05:04:33 - INFO - finbert.utils -   input_ids: 101 25022 3775 1005 1055 2327 15768 2005 2502 11443 4859 3930 1999 16798 2475 1024 6541 9887 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
01/18/2022 05:04:33 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
01/18/2022 05:04:33 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
01/18/2022 05:04:33 - INFO - finbert.utils -   label: None (id = 9090)
01/18/2022 05:04:34 - INFO - root -   te

In [12]:
headlines_sentiment_analysis_result['score'][5].mean()

0.11958521604537964

In [13]:
headlines_sentiment_analysis_result['score'] = [scores.mean() 
                    for scores in headlines_sentiment_analysis_result['score']]

In [14]:
headlines_sentiment_analysis_result

{'date': ['2021-12-18',
  '2021-12-17',
  '2021-12-17',
  '2021-12-16',
  '2021-12-15',
  '2021-12-15',
  '2021-12-15',
  '2021-12-14',
  '2021-12-14',
  '2021-12-13',
  '2021-12-10',
  '2021-12-10',
  '2021-12-09',
  '2021-12-08',
  '2021-12-08',
  '2021-12-07',
  '2021-12-06',
  '2021-12-06',
  '2021-12-05',
  '2021-12-04',
  '2021-12-03',
  '2021-12-02',
  '2021-12-02',
  '2021-12-02',
  '2021-12-01',
  '2021-12-01',
  '2021-11-30',
  '2021-11-30',
  '2021-11-29',
  '2021-11-29',
  '2021-11-29',
  '2021-11-29',
  '2021-11-26',
  '2021-11-26',
  '2021-11-24',
  '2021-11-24',
  '2021-11-23',
  '2021-11-22',
  '2021-11-22',
  '2021-11-22',
  '2021-11-21',
  '2021-11-19',
  '2021-11-18',
  '2021-11-17',
  '2021-11-16',
  '2021-11-16',
  '2021-11-15',
  '2021-11-12',
  '2021-11-12',
  '2021-11-12',
  '2021-11-11',
  '2021-11-10',
  '2021-11-10',
  '2021-11-09',
  '2021-11-09',
  '2021-11-08',
  '2021-11-05',
  '2021-11-04',
  '2021-11-03',
  '2021-11-02',
  '2021-11-02',
  '2021-11-02',


In [15]:
pd.DataFrame.from_dict(headlines_sentiment_analysis_result).to_csv('headlines_score.csv', index=False)

In [21]:
headlines_score_df = pd.read_csv('headlines_score.csv')
headlines_score_df.head()

,date,score
0,2021-12-18,0.315013
1,2021-12-17,-0.943284
2,2021-12-17,-0.547781
3,2021-12-16,-0.952843
4,2021-12-15,0.494560


In [26]:
headlines_score_df.groupby(headlines_score_df.date).mean().to_csv('final_news_score.csv')